In [1]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])

# Import the backtrader platform
import backtrader as bt
import backtrader.indicators as btind

from backtrader.feeds import GenericCSVData

from statistics import mean
from statistics import median

In [2]:
class dix_csv(GenericCSVData):
    
    lines= ('dix', 'gex')
    
    params = (
        ('dtformat', '%m/%d/%Y'),
        ('datetime', 0),
        ('time', -1),
        ('open', 1),
        ('high', 2),
        ('low', 3),
        ('close', 4),
        ('volume', 6),
        ('openinterest', -1),
        ('dix', 9),
        ('gex', 10),        
    )

In [ ]:
#Optimize DIX MA
class DIXmaopt(bt.Strategy):
    params = (
        ('period', 2),
        ('printlog', False),
        ('dixupper', 45),
        ('dixlower', 40),
    )
    def start(self):
        self.val_start = self.broker.get_cash()  # keep the starting cash

    def log(self, txt, dt=None, doprint=False):
        ''' Logging function fot this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

        #Add a DIXSMA Indicator
        self.dixsma = bt.indicators.MovingAverageSimple(
           self.data.dix, period=self.params.period)


    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
        #make a list of the pnl for each trade
        self.pnl_percent = trade.pnl / self.broker.getvalue()
        self.pnl_list=[]
        self.pnl_list.append(self.pnl_percent)

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # buy when dix greater than dixupper threshold
            if self.dixsma >= (self.params.dixupper/100):
                self.log('BUY CREATE, %.2f' % self.dataclose[0])
                self.order = self.buy()
                
        else:
            
            # sell after 60 trading days
            self.duration = len(self) - self.bar_executed + 1
            if self.duration >= 60:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()
    def stop(self):
        #roi
        self.roi = (self.broker.get_value() / self.val_start) - 1.0
        #take the mean if a pnl exists
        try:
            self.pnl_mean = (mean(self.pnl_list) * 100)
        except:
            self.pnl_list = [0]
            self.pnl_mean = (mean(self.pnl_list) * 100)
        #take the median if a pnl exists    
        try:
            self.pnl_median = (median(self.pnl_list) * 100)
        except:
            self.pnl_list = [0]
            self.pnl_median = (median(self.pnl_list) * 100)
            
        print('(MA Period%2d) (DIXUpper %2d) (MEAN TRADE PNL: %.3f%%) (MEDIAN TRADE PNL: %.3f%%) ROI %.2f%%' %
                 (self.params.period, self.params.dixupper, self.pnl_mean, self.pnl_median, self.roi))
        
#runDIXma Strategy Opt
if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()    
    
    # Add a strategy
    strats = cerebro.optstrategy(
        DIXmaopt,
        period=range(1,501),
        dixupper=range(41,51)
    ) 
    
    
    # Datas are in a subfolder of the samples. Need to find where the script is
    # because it could have been called from anywhere
    modpath = os.path.dirname(os.path.abspath(sys.argv[0]))
    datapath = os.path.join(modpath, 'C:\\Users\\big_u\\DIX MA Testing Backtrader\DIX OHLC.csv')

    # Create a Data Feed
    data = dix_csv(
        dataname=datapath,
    )

    # Add the Data Feed to Cerebro
    cerebro.adddata(data)

    # Set our desired cash start
    cerebro.broker.setcash(100000.0)

    # Add a FixedSize sizer according to the stake
    cerebro.addsizer(bt.sizers.FixedSize, stake=1)

    # Print out the starting conditions
    #print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run(maxcpus=1)
    
    # Set the commission
    cerebro.broker.setcommission(commission=0.00)

    # Print out the final result
    #print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())


(MA Period 1) (DIXUpper 41) (MEAN TRADE PNL: -0.440%) (MEDIAN TRADE PNL: -0.440%) ROI 0.02%
(MA Period 1) (DIXUpper 42) (MEAN TRADE PNL: -0.432%) (MEDIAN TRADE PNL: -0.432%) ROI 0.02%
(MA Period 1) (DIXUpper 43) (MEAN TRADE PNL: -0.454%) (MEDIAN TRADE PNL: -0.454%) ROI 0.02%
(MA Period 1) (DIXUpper 44) (MEAN TRADE PNL: -0.049%) (MEDIAN TRADE PNL: -0.049%) ROI 0.02%
(MA Period 1) (DIXUpper 45) (MEAN TRADE PNL: 0.583%) (MEDIAN TRADE PNL: 0.583%) ROI 0.02%
(MA Period 1) (DIXUpper 46) (MEAN TRADE PNL: 0.584%) (MEDIAN TRADE PNL: 0.584%) ROI 0.02%
(MA Period 1) (DIXUpper 47) (MEAN TRADE PNL: 0.575%) (MEDIAN TRADE PNL: 0.575%) ROI 0.01%
(MA Period 1) (DIXUpper 48) (MEAN TRADE PNL: 0.574%) (MEDIAN TRADE PNL: 0.574%) ROI 0.02%
(MA Period 1) (DIXUpper 49) (MEAN TRADE PNL: 0.568%) (MEDIAN TRADE PNL: 0.568%) ROI 0.01%
(MA Period 1) (DIXUpper 50) (MEAN TRADE PNL: 0.556%) (MEDIAN TRADE PNL: 0.556%) ROI 0.01%
(MA Period 2) (DIXUpper 41) (MEAN TRADE PNL: -0.432%) (MEDIAN TRADE PNL: -0.432%) ROI 0.02%


(MA Period10) (DIXUpper 42) (MEAN TRADE PNL: 0.557%) (MEDIAN TRADE PNL: 0.557%) ROI 0.02%
(MA Period10) (DIXUpper 43) (MEAN TRADE PNL: 0.545%) (MEDIAN TRADE PNL: 0.545%) ROI 0.03%
(MA Period10) (DIXUpper 44) (MEAN TRADE PNL: 0.557%) (MEDIAN TRADE PNL: 0.557%) ROI 0.01%
(MA Period10) (DIXUpper 45) (MEAN TRADE PNL: 0.526%) (MEDIAN TRADE PNL: 0.526%) ROI 0.02%
(MA Period10) (DIXUpper 46) (MEAN TRADE PNL: 0.522%) (MEDIAN TRADE PNL: 0.522%) ROI 0.01%
(MA Period10) (DIXUpper 47) (MEAN TRADE PNL: 0.403%) (MEDIAN TRADE PNL: 0.403%) ROI 0.00%
(MA Period10) (DIXUpper 48) (MEAN TRADE PNL: 0.081%) (MEDIAN TRADE PNL: 0.081%) ROI 0.01%
(MA Period10) (DIXUpper 49) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period10) (DIXUpper 50) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period11) (DIXUpper 41) (MEAN TRADE PNL: -0.449%) (MEDIAN TRADE PNL: -0.449%) ROI 0.02%
(MA Period11) (DIXUpper 42) (MEAN TRADE PNL: 0.557%) (MEDIAN TRADE PNL: 0.557%) ROI 0.02%
(MA Peri

(MA Period19) (DIXUpper 43) (MEAN TRADE PNL: -0.653%) (MEDIAN TRADE PNL: -0.653%) ROI 0.01%
(MA Period19) (DIXUpper 44) (MEAN TRADE PNL: 0.298%) (MEDIAN TRADE PNL: 0.298%) ROI 0.02%
(MA Period19) (DIXUpper 45) (MEAN TRADE PNL: 0.296%) (MEDIAN TRADE PNL: 0.296%) ROI 0.01%
(MA Period19) (DIXUpper 46) (MEAN TRADE PNL: 0.118%) (MEDIAN TRADE PNL: 0.118%) ROI 0.00%
(MA Period19) (DIXUpper 47) (MEAN TRADE PNL: 0.081%) (MEDIAN TRADE PNL: 0.081%) ROI 0.00%
(MA Period19) (DIXUpper 48) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period19) (DIXUpper 49) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period19) (DIXUpper 50) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period20) (DIXUpper 41) (MEAN TRADE PNL: -0.442%) (MEDIAN TRADE PNL: -0.442%) ROI 0.01%
(MA Period20) (DIXUpper 42) (MEAN TRADE PNL: 0.552%) (MEDIAN TRADE PNL: 0.552%) ROI 0.02%
(MA Period20) (DIXUpper 43) (MEAN TRADE PNL: -0.679%) (MEDIAN TRADE PNL: -0.679%) ROI 0.01%
(MA 

(MA Period28) (DIXUpper 44) (MEAN TRADE PNL: 0.258%) (MEDIAN TRADE PNL: 0.258%) ROI 0.01%
(MA Period28) (DIXUpper 45) (MEAN TRADE PNL: 0.097%) (MEDIAN TRADE PNL: 0.097%) ROI 0.01%
(MA Period28) (DIXUpper 46) (MEAN TRADE PNL: 0.094%) (MEDIAN TRADE PNL: 0.094%) ROI 0.00%
(MA Period28) (DIXUpper 47) (MEAN TRADE PNL: 0.021%) (MEDIAN TRADE PNL: 0.021%) ROI 0.00%
(MA Period28) (DIXUpper 48) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period28) (DIXUpper 49) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period28) (DIXUpper 50) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period29) (DIXUpper 41) (MEAN TRADE PNL: -0.282%) (MEDIAN TRADE PNL: -0.282%) ROI 0.01%
(MA Period29) (DIXUpper 42) (MEAN TRADE PNL: 0.527%) (MEDIAN TRADE PNL: 0.527%) ROI 0.02%
(MA Period29) (DIXUpper 43) (MEAN TRADE PNL: -0.466%) (MEDIAN TRADE PNL: -0.466%) ROI 0.01%
(MA Period29) (DIXUpper 44) (MEAN TRADE PNL: 0.257%) (MEDIAN TRADE PNL: 0.257%) ROI 0.01%
(MA Pe

(MA Period37) (DIXUpper 45) (MEAN TRADE PNL: 0.128%) (MEDIAN TRADE PNL: 0.128%) ROI 0.01%
(MA Period37) (DIXUpper 46) (MEAN TRADE PNL: -0.084%) (MEDIAN TRADE PNL: -0.084%) ROI 0.00%
(MA Period37) (DIXUpper 47) (MEAN TRADE PNL: 0.139%) (MEDIAN TRADE PNL: 0.139%) ROI 0.00%
(MA Period37) (DIXUpper 48) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period37) (DIXUpper 49) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period37) (DIXUpper 50) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period38) (DIXUpper 41) (MEAN TRADE PNL: -0.168%) (MEDIAN TRADE PNL: -0.168%) ROI 0.02%
(MA Period38) (DIXUpper 42) (MEAN TRADE PNL: 0.521%) (MEDIAN TRADE PNL: 0.521%) ROI 0.01%
(MA Period38) (DIXUpper 43) (MEAN TRADE PNL: -0.369%) (MEDIAN TRADE PNL: -0.369%) ROI 0.01%
(MA Period38) (DIXUpper 44) (MEAN TRADE PNL: 0.165%) (MEDIAN TRADE PNL: 0.165%) ROI 0.02%
(MA Period38) (DIXUpper 45) (MEAN TRADE PNL: 0.126%) (MEDIAN TRADE PNL: 0.126%) ROI 0.01%
(MA 

...

(MA Period452) (DIXUpper 45) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period452) (DIXUpper 46) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period452) (DIXUpper 47) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period452) (DIXUpper 48) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period452) (DIXUpper 49) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period452) (DIXUpper 50) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period453) (DIXUpper 41) (MEAN TRADE PNL: -0.168%) (MEDIAN TRADE PNL: -0.168%) ROI 0.02%
(MA Period453) (DIXUpper 42) (MEAN TRADE PNL: -0.448%) (MEDIAN TRADE PNL: -0.448%) ROI 0.01%
(MA Period453) (DIXUpper 43) (MEAN TRADE PNL: 0.196%) (MEDIAN TRADE PNL: 0.196%) ROI 0.01%
(MA Period453) (DIXUpper 44) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period453) (DIXUpper 45) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0

(MA Period461) (DIXUpper 45) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period461) (DIXUpper 46) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period461) (DIXUpper 47) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period461) (DIXUpper 48) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period461) (DIXUpper 49) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period461) (DIXUpper 50) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period462) (DIXUpper 41) (MEAN TRADE PNL: -0.281%) (MEDIAN TRADE PNL: -0.281%) ROI 0.02%
(MA Period462) (DIXUpper 42) (MEAN TRADE PNL: -0.113%) (MEDIAN TRADE PNL: -0.113%) ROI 0.01%
(MA Period462) (DIXUpper 43) (MEAN TRADE PNL: 0.233%) (MEDIAN TRADE PNL: 0.233%) ROI 0.01%
(MA Period462) (DIXUpper 44) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period462) (DIXUpper 45) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0

(MA Period470) (DIXUpper 45) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period470) (DIXUpper 46) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period470) (DIXUpper 47) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period470) (DIXUpper 48) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period470) (DIXUpper 49) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period470) (DIXUpper 50) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period471) (DIXUpper 41) (MEAN TRADE PNL: -0.281%) (MEDIAN TRADE PNL: -0.281%) ROI 0.02%
(MA Period471) (DIXUpper 42) (MEAN TRADE PNL: 0.562%) (MEDIAN TRADE PNL: 0.562%) ROI 0.01%
(MA Period471) (DIXUpper 43) (MEAN TRADE PNL: 0.289%) (MEDIAN TRADE PNL: 0.289%) ROI 0.01%
(MA Period471) (DIXUpper 44) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period471) (DIXUpper 45) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.0

(MA Period479) (DIXUpper 45) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period479) (DIXUpper 46) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period479) (DIXUpper 47) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period479) (DIXUpper 48) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period479) (DIXUpper 49) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period479) (DIXUpper 50) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period480) (DIXUpper 41) (MEAN TRADE PNL: -0.168%) (MEDIAN TRADE PNL: -0.168%) ROI 0.02%
(MA Period480) (DIXUpper 42) (MEAN TRADE PNL: 0.752%) (MEDIAN TRADE PNL: 0.752%) ROI 0.01%
(MA Period480) (DIXUpper 43) (MEAN TRADE PNL: 0.104%) (MEDIAN TRADE PNL: 0.104%) ROI 0.00%
(MA Period480) (DIXUpper 44) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period480) (DIXUpper 45) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.0

(MA Period488) (DIXUpper 45) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period488) (DIXUpper 46) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period488) (DIXUpper 47) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period488) (DIXUpper 48) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period488) (DIXUpper 49) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period488) (DIXUpper 50) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period489) (DIXUpper 41) (MEAN TRADE PNL: -0.113%) (MEDIAN TRADE PNL: -0.113%) ROI 0.02%
(MA Period489) (DIXUpper 42) (MEAN TRADE PNL: 0.572%) (MEDIAN TRADE PNL: 0.572%) ROI 0.01%
(MA Period489) (DIXUpper 43) (MEAN TRADE PNL: 0.120%) (MEDIAN TRADE PNL: 0.120%) ROI 0.01%
(MA Period489) (DIXUpper 44) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.00%
(MA Period489) (DIXUpper 45) (MEAN TRADE PNL: 0.000%) (MEDIAN TRADE PNL: 0.000%) ROI 0.0